# Pairwise Experiments

### Setup

In [2]:
!pip install --quiet -U langchain-google-genai langgraph langgraph-sdk langgraph-checkpoint-sqlite langsmith langchain-community langchain-core
!pip install --quiet notebook python-dotenv lxml scikit-learn pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216

In [3]:
!pip install --quiet python-dotenv

In [4]:
from dotenv import load_dotenv
load_dotenv(".env")

True

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Task

Let's set up a new task! Here, we have a salesperson named Bob. Bob has a lot of deals, so he wants to summarize what happened in this deals based off of some meeting transcripts.

Bob is iterating on a few different prompts, that will give him nice, concise transcripts for his deals.

Bob has curated a dataset of his deal transcripts, let's go ahead and load that in. You can take a look at the dataset as well if you're curious! Note that this is not a golden dataset, there is no reference output here.

In [9]:
from langsmith import Client

ls_client = Client()
dataset = ls_client.clone_public_dataset(
  "https://smith.langchain.com/public/9078d2f1-7bef-4ba7-b795-210a17682ef9/d"
)

### Experiments

Now, let's run some experiments on this dataset using two different prompts. Let's add an evaluator that tries to score how good our summaries are!

In [10]:
from pydantic import BaseModel, Field
#from openai import OpenAI
import json

#openai_client = OpenAI()
import os
from google import genai
# openai_client = OpenAI()
g_client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))

SUMMARIZATION_SYSTEM_PROMPT = """You are a judge, aiming to score how well a summary summarizes the content of a transcript"""

SUMMARIZATION_HUMAN_PROMPT = """
[The Meeting Transcript] {transcript}
[The Start of Summarization] {summary} [The End of Summarization]"""

class SummarizationScore(BaseModel):
    score: int = Field(description="""A score from 1-5 ranking how good the summarization is for the provided transcript, with 1 being a bad summary, and 5 being a great summary""")

def summary_score_evaluator(inputs: dict, outputs: dict) -> list:
    # completion = openai_client.beta.chat.completions.parse(
    #     model="gpt-4o",
    #     messages=[
    #         {
    #             "role": "system",
    #             "content": SUMMARIZATION_SYSTEM_PROMPT,
    #         },
    #         {
    #             "role": "user",
    #             "content": SUMMARIZATION_HUMAN_PROMPT.format(
    #                 transcript=inputs["transcript"],
    #                 summary=outputs.get("output", "N/A"),
    #             )}
    #     ],
    #     response_format=SummarizationScore,
    # )

    # summary_score = completion.choices[0].message.parsed.score
    # return {"key": "summary_score", "score": summary_score}

    completion = g_client.models.generate_content(
    model="gemini-2.5-flash-lite", contents=[
             {
                      "role": "user",
                      "parts": [
                          {"text": SUMMARIZATION_SYSTEM_PROMPT},
                          {"text":  SUMMARIZATION_HUMAN_PROMPT.format(
                    transcript=inputs["transcript"],
                    summary=outputs.get("output", "N/A"),
                )}
                      ]
              }],
             config={
        "response_mime_type": "application/json",
        "response_schema": SummarizationScore,
    },
    )
    # Your JSON string from the API response
    json_string = completion.candidates[0].content.parts[0].text

    # Use json.loads() to parse the string into a Python dictionary
    data_dictionary = json.loads(json_string)

    # Now access the value using its key
    summary_score = data_dictionary["score"]

    # print(similarity_score) # This will also print: 1
    return {"score": summary_score, "key": "summary"}

First, we'll run our experiment with a good version of our prompt!

In [11]:
# Prompt One: Good Prompt!
def good_summarizer(inputs: dict):
    # response = openai_client.chat.completions.create(
    #     model="gpt-4o",
    #     messages=[
    #         {
    #             "role": "user",
    #             "content": f"Concisely summarize this meeting in 3 sentences. Make sure to include all of the important events. Meeting: {inputs['transcript']}"
    #         }
    #     ],
    # )
    # return response.choices[0].message.content

    completion = g_client.models.generate_content(
    model="gemini-2.5-flash-lite", contents=[
             {
                      "role": "user",
                      "parts": [
                          {"text": f"Concisely summarize this meeting in 3 sentences. Make sure to include all of the important events. Meeting: {inputs['transcript']}"}
                      ]
              }],
    )
    # Your JSON string from the API response
    return completion.candidates[0].content.parts[0].text

ls_client.evaluate(
    good_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Good Summarizer"
)

View the evaluation results for experiment: 'Good Summarizer-de192aa3' at:
https://smith.langchain.com/o/10172e86-d29b-46ce-8113-9bee95385ee1/datasets/9caace53-eeb2-4262-9035-da62e884fe11/compare?selectedSessions=eb90a0dc-0c78-4833-b234-4fa3d71a6da4




0it [00:00, ?it/s]

,inputs.transcript,outputs.output,error,feedback.summary,execution_time,example_id,id
0,"Bob and Mr. Patel (CLOSED DEAL): Bob: Hello, M...",Bob successfully sold a Ford Fusion Hybrid to ...,None,5,0.640767,1aee2497-a4b5-4b80-bbb8-c04959207ebe,7bdfdcdf-b98e-461d-bba9-847cac1597cb
1,Bob and Ms. Nguyen (NO DEAL): Bob: Good aftern...,"Bob met with Ms. Nguyen, who was looking for a...",None,4,0.502531,55fff590-da65-4ef7-bd9f-75a6cd12a74b,4483825b-fa67-4de2-9f30-032f94b5d4a0
2,Bob and Mr. Johnson (CLOSED DEAL): Bob: Good m...,Mr. Johnson visited Ford Motors seeking a spac...,None,5,0.714914,6ae50f14-995c-4ec1-96b1-68529427df2f,dde3b33b-c1ec-4d81-85a2-dbd35f34d6a8
3,Bob and Mr. Carter (CLOSED DEAL): Bob: Welcome...,Mr. Carter visited Bob to trade in his 2015 Ta...,None,5,0.516808,7c9abeb9-aff4-4eed-9ed8-1ff6ac66ea8d,bc4e08e1-2d76-4f04-b86d-1ebcf619f5a7
4,"Bob and Ms. Thompson (NO DEAL): Bob: Hi, Ms. T...",Ms. Thompson visited Ford Motors to browse SUV...,None,5,0.573581,fa11dc6d-92d4-4a49-9da3-47315bcc0d26,c4a7597a-2a93-4a27-a7f4-55783a800b80


Now, we'll run an experiment with a worse version of our prompt, to highlight the difference.

In [12]:
# Prompt Two: Worse Prompt!
def bad_summarizer(inputs: dict):
    # response = openai_client.chat.completions.create(
    #     model="gpt-4o",
    #     messages=[
    #         {
    #             "role": "user",
    #             "content": f"Summarize this in one sentence. {inputs['transcript']}"
    #         }
    #     ],
    # )
    # return response.choices[0].message.content
    completion = g_client.models.generate_content(
    model="gemini-2.5-flash-lite", contents=[
             {
                      "role": "user",
                      "parts": [
                          {"text":  f"Summarize this in one sentence. {inputs['transcript']}"}
                      ]
              }],
    )
    # Your JSON string from the API response
    return completion.candidates[0].content.parts[0].text

ls_client.evaluate(
    bad_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Bad Summarizer"
)

View the evaluation results for experiment: 'Bad Summarizer-67af22af' at:
https://smith.langchain.com/o/10172e86-d29b-46ce-8113-9bee95385ee1/datasets/9caace53-eeb2-4262-9035-da62e884fe11/compare?selectedSessions=e9f0cec4-2f32-45ab-8500-52e32670f9db




0it [00:00, ?it/s]

,inputs.transcript,outputs.output,error,feedback.summary,execution_time,example_id,id
0,"Bob and Mr. Patel (CLOSED DEAL): Bob: Hello, M...",Bob successfully sold a Ford Fusion Hybrid to ...,None,5,0.723219,1aee2497-a4b5-4b80-bbb8-c04959207ebe,726e6991-7c91-4ed7-9ad9-d06504000ac0
1,Bob and Ms. Nguyen (NO DEAL): Bob: Good aftern...,"Despite Bob's suggestions of the Ford Fiesta, ...",None,4,0.418863,55fff590-da65-4ef7-bd9f-75a6cd12a74b,9d5af1ed-6846-4b3d-a733-de873ff158d7
2,Bob and Mr. Johnson (CLOSED DEAL): Bob: Good m...,Bob successfully sold a Ford Explorer to Mr. J...,None,5,0.414092,6ae50f14-995c-4ec1-96b1-68529427df2f,83be7c05-445f-4ea4-8e31-aa2283969621
3,Bob and Mr. Carter (CLOSED DEAL): Bob: Welcome...,Bob successfully guided Mr. Carter from expres...,None,5,0.450759,7c9abeb9-aff4-4eed-9ed8-1ff6ac66ea8d,cec6fa8d-4276-42cf-a7b0-485d6753ebfc
4,"Bob and Ms. Thompson (NO DEAL): Bob: Hi, Ms. T...",Ms. Thompson visited Ford Motors to browse SUV...,None,4,0.519285,fa11dc6d-92d4-4a49-9da3-47315bcc0d26,740edb75-b59e-4f3e-ad6f-dc5153603f98


### Pairwise Experiment

Let's define a function that will compare our two experiments. These are the fields that pairwise evaluator functions get access to:
- `inputs: dict`: A dictionary of the inputs corresponding to a single example in a dataset.
- `outputs: list[dict]`: A list of the dict outputs produced by each experiment on the given inputs.
- `reference_outputs: dict`: A dictionary of the reference outputs associated with the example, if available.
- `runs: list[Run]`: A list of the full Run objects generated by the experiments on the given example. Use this if you need access to intermediate steps or metadata about each run.
- `example: Example`: The full dataset Example, including the example inputs, outputs (if available), and metdata (if available).

First, let's give our LLM-as-Judge some instructions. In our case, we're just going to directly use LLM-as-judge to grade which of the summarizers is the most helpful.

It might be hard to grade our summarizers without a ground truth reference, but here, comparing different prompts head to head will give us a sense of which is better!

In [13]:
JUDGE_SYSTEM_PROMPT = """
Please act as an impartial judge and evaluate the quality of the summarizations provided by two AI summarizers to the meeting transcript below.
Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their summarizations.
Begin your evaluation by comparing the two summarizations and provide a short explanation.
Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision.
Do not favor certain names of the assistants.
Be as objective as possible. """

JUDGE_HUMAN_PROMPT = """
[The Meeting Transcript] {transcript}

[The Start of Assistant A's Summarization] {answer_a} [The End of Assistant A's Summarization]

[The Start of Assistant B's Summarization] {answer_b} [The End of Assistant B's Summarization]"""

Our function will take in an `inputs` dictionary, and a list of `outputs` dictionaries for the different experiments that we want to compare.

In [23]:
from pydantic import BaseModel, Field

class Preference(BaseModel):
    preference: int = Field(description="""1 if Assistant A answer is better based upon the factors above.
2 if Assistant B answer is better based upon the factors above.
Output 0 if it is a tie.""")

def ranked_preference(inputs: dict, outputs: list[dict]) -> list:
    # completion = openai_client.beta.chat.completions.parse(
    #     model="gpt-4o",
    #     messages=[
    #         {
    #             "role": "system",
    #             "content": JUDGE_SYSTEM_PROMPT,
    #         },
    #         {
    #             "role": "user",
    #             "content": JUDGE_HUMAN_PROMPT.format(
    #                 transcript=inputs["transcript"],
    #                 answer_a=outputs[0].get("output", "N/A"),
    #                 answer_b=outputs[1].get("output", "N/A")
    #             )}
    #     ],
    #     response_format=Preference,
    # )

    completion = g_client.models.generate_content(
    model="gemini-2.5-flash-lite", contents=[
             {
                      "role": "user",
                      "parts": [
                          {"text": JUDGE_SYSTEM_PROMPT},
                          {"text": JUDGE_HUMAN_PROMPT.format(
                    transcript=inputs["transcript"],
                    answer_a=outputs[0].get("output", "N/A"),
                    answer_b=outputs[1].get("output", "N/A")
                )}
                      ]
              }],
             config={
        "response_mime_type": "application/json",
        "response_schema": Preference,
    },
    )
    # Your JSON string from the API response
    json_string = completion.candidates[0].content.parts[0].text
    # print(json_string)
    # Use json.loads() to parse the string into a Python dictionary
    data_dictionary = json.loads(json_string)

    # Now access the value using its key
    preference_score = data_dictionary["preference"]


    # preference_score = completion.choices[0].message.parsed.preference

    if preference_score == 1:
        scores = [1, 0]
    elif preference_score == 2:
        scores = [0, 1]
    else:
        scores = [0, 0]
    return scores

Now let's run our pairwise experiment with `evaluate()`

In [24]:
from langsmith import evaluate

evaluate(
    ("Good Summarizer-de192aa3", "Bad Summarizer-67af22af"),  # TODO: Replace with the names/IDs of your experiments
    evaluators=[ranked_preference]
)

View the pairwise evaluation results at:
https://smith.langchain.com/o/10172e86-d29b-46ce-8113-9bee95385ee1/datasets/9caace53-eeb2-4262-9035-da62e884fe11/compare?selectedSessions=eb90a0dc-0c78-4833-b234-4fa3d71a6da4%2Ce9f0cec4-2f32-45ab-8500-52e32670f9db&comparativeExperiment=8dfd609f-1859-469f-864c-a7d65d24ece4




  0%|          | 0/5 [00:00<?, ?it/s]